In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
from PopulationMode import PopulationModeModel

"""
Test for simulating population mode; simulate both models and compare the 
result by plotting their result

"""
tau_ref = 0.0025 # refactory time
tau_RC = 0.02 # membrane RC time constant
n = 200
intercepts = -1 + 2*np.random.rand(1,n)[0]
max_rates = 80 + 50*np.random.rand(1,n)[0]
v0 = 0
dt = 0.0005

model = nengo.Network(label="test_populationMode")
with model:
    # Default population
    pop2D = nengo.Ensemble(n, dimensions=2, intercepts=intercepts,
            max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
            label="2D Population")
    orig_input = nengo.Node(lambda x: x) # input to the population
    print ("dimension is: {0}".format(pop2D.dimensions))
    # Add connections
    orig_conn_1 = nengo.Connection(orig_input, pop2D[0])
    orig_conn_2 = nengo.Connection(orig_input, pop2D[1])

    # Surrogate population
    surr_pop2D = PopulationModeModel(pop2D)
    surr_pop2D.createBiasModel()
    # surr_pop2D.createNoiseModel()

    orig_probe = nengo.Probe(orig_input)

# Simulate the model for second(s)
with nengo.Simulator(model) as sim:
    sim.run(2)

def_points = pop2D.eval_points
pop_rates = pop2D.neuron_type.rates(sim.data[orig_probe], 
        gain=sim.data[pop2D].gain, bias=sim.data[pop2D].bias)
popMode_rates = surr_pop2D.pop.neuron_type.rates(sim.data[orig_probe], gain=sim.data[surr_pop2D].gain, bias=sim.data[surr_pop2D].bias)

plt.figure()
plt.title('Firing rate changing over time with default mode')
plt.plot(def_rates)
plt.ylabel('Firing rate (Hz)')
plt.xlabel('Time (ms)')
plt.savefig('Firing rate chaging over time.png')

plt.figure()
plt.title('Firing rate changing over time with population mode')
plt.plot(pop_rates)
plt.ylabel('Firing rate (Hz)')
plt.xlabel('Time (ms)')
plt.savefig('Firing rate chaging over time.png')

# [simBias, ideal] = surr_pop2D.getBiasSamples(points)
# modelBias = np.zeros(simBias.shape)
# diffBias = np.zeros(simBias.shape)
# diffThresh = 0

# for i in range(n):
#     modelBias[:,i] = pop.getBias(points[:,i])
#     diffBias[:,i] = modelBias[:,i] - simBias[:,i]


dimension is: 2
originIndices.shape: (1, 2)
len(originIndices): 1


TypeError: object of type 'int' has no len()